<a href="https://colab.research.google.com/github/Buzaabah/livelossplot/blob/master/NL2SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd original\ copy

In [0]:
#Important parckages

%matplotlib inline

import re,string
import numpy as np
import pickle
import spacy
from spacy.symbols import ORTH

en_tok = spacy.load('en')

from collections import Counter,defaultdict
from pathlib import Path


In [53]:
#Reading required data

train_data = open('atis.uw.train.txt').readlines()
train_data[:5]
#len(train_data)

["list all the flights that arrive at general mitchell international from various cities ||| SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport airport_1 , airport_service airport_service_1 , city city_1 WHERE flight_1.to_airport = airport_1.airport_code AND airport_1.airport_code = 'MKE' AND flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code\n",
 "give me the flights leaving denver august ninth coming back to boston ||| SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport_service airport_service_1 , city city_1 , airport_service airport_service_2 , city city_2 , days days_1 , date_day date_day_1 WHERE flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code AND city_1.city_name = 'DENVER' AND ( flight_1.to_airport = airport_service_2.airport_code AND airport_service_2.city_code = city_2.city_code AND city_2.city_name = 'BOSTON' AND flight_1.flight_days = 

In [51]:
test_data = open('atis.uw.test.txt').readlines()
test_data[:5]
len(test_data)

447

In [52]:
dev_data = open('atis.uw.dev.txt').readlines()
dev_data[:5]

len(dev_data)

486

In [0]:
# Combining train, dev and test sets

file_list = ['atis.uw.train.txt', 'atis.uw.dev.txt', 'atis.uw.test.txt']
output_file = Path('Out_file.txt')
for file in file_list:
  output_file.write_text(Path(file).read_text())

In [50]:
# Reading a combined file

data = open('Out_file.txt').readlines()
#data[:5]
len(data)

447

In [0]:
# making a corpus

def make_corpus(corpus_path):
    en,sql = [],[]
    for line in train_data:
        if '\n' in line:
            new_line = line.replace('\n','')
            new_line = new_line.split('|||')
            en.append(new_line[0])
            sql.append(new_line[1])
    return en,sql

In [16]:
#unpacking the corpus

en, sql = make_corpus('atis.uw.train.txt')
en[:2],sql[:2]

(['list all the flights that arrive at general mitchell international from various cities ',
  'give me the flights leaving denver august ninth coming back to boston '],
 [" SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport airport_1 , airport_service airport_service_1 , city city_1 WHERE flight_1.to_airport = airport_1.airport_code AND airport_1.airport_code = 'MKE' AND flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code",
  " SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport_service airport_service_1 , city city_1 , airport_service airport_service_2 , city city_2 , days days_1 , date_day date_day_1 WHERE flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code AND city_1.city_name = 'DENVER' AND ( flight_1.to_airport = airport_service_2.airport_code AND airport_service_2.city_code = city_2.city_code AND city_2.city_name = 'BOSTON' AND flight_1.flight_

In [0]:
#pre-processing for tokenizing 
# Natural language tokens

def en_tokenizer(text):
    text = text.lower()
    return [t.text for t in en_tok.tokenizer(text)]
  
#SQL tokens
def en_tokenizer(text):
    text = text
    return [t.text for t in en_tok.tokenizer(text)]

In [18]:
# Tokens

en_toks = [en_tokenizer(text) for text in en]
sql_toks = [en_tokenizer(text) for text in sql]
en_toks[:2], sql_toks[:2]

([['list',
   'all',
   'the',
   'flights',
   'that',
   'arrive',
   'at',
   'general',
   'mitchell',
   'international',
   'from',
   'various',
   'cities'],
  ['give',
   'me',
   'the',
   'flights',
   'leaving',
   'denver',
   'august',
   'ninth',
   'coming',
   'back',
   'to',
   'boston']],
 [[' ',
   'SELECT',
   'DISTINCT',
   'flight_1.flight_id',
   'FROM',
   'flight',
   'flight_1',
   ',',
   'airport',
   'airport_1',
   ',',
   'airport_service',
   'airport_service_1',
   ',',
   'city',
   'city_1',
   'WHERE',
   'flight_1.to_airport',
   '=',
   'airport_1.airport_code',
   'AND',
   'airport_1.airport_code',
   '=',
   "'",
   'MKE',
   "'",
   'AND',
   'flight_1.from_airport',
   '=',
   'airport_service_1.airport_code',
   'AND',
   'airport_service_1.city_code',
   '=',
   'city_1.city_code'],
  [' ',
   'SELECT',
   'DISTINCT',
   'flight_1.flight_id',
   'FROM',
   'flight',
   'flight_1',
   ',',
   'airport_service',
   'airport_service_1',
   ',

In [19]:
len(en_toks), len(sql_toks)

(4350, 4350)

In [0]:
# Creating a dictionary/numerical order

def numericalize_tok(tokens, max_vocab=10000, min_freq=0, unk_tok="xxunk", pad_tok="xxpad", bos_tok="xxbos", eos_tok="xxeos"):
    if isinstance(tokens, str):
        raise ValueError("Expected to receive a list of tokens. Received a string instead")
    if isinstance(tokens[0], list):
        tokens = [v for w in tokens for v in w]
    freq = Counter(tokens)
    int2tok = [w for w,z in freq.most_common(max_vocab) if z>min_freq]
    unk_id = 3
    int2tok.insert(0, bos_tok)
    int2tok.insert(1, pad_tok)
    int2tok.insert(2, eos_tok)
    int2tok.insert(unk_id, unk_tok)
    tok2int = defaultdict(lambda:unk_id, {n:k for k,n in enumerate(int2tok)})
    return int2tok, tok2int
  
 

In [0]:
int2sql,sql2int = numericalize_tok(sql_toks)
int2en,en2int = numericalize_tok(en_toks)

In [23]:
len(int2sql), len(int2en)

(688, 859)

In [0]:
ls

In [0]:
# serializing (Saving) a list of tokens for further usage 
# english tokens

int2en_out = open("int2en.pkl","wb")
pickle.dump(int2en, int2en_out)
int2en_out.close()

In [0]:
# serializing (Saving) a list of tokens for further usage 
# SQL tokens

int2sql_out = open("int2sql.pkl","wb")
pickle.dump(int2sql, int2sql_out)
int2sql_out.close()

In [0]:
# Reading a list of tokens for further usage 
# English tokens
  
  with open ('int2en.pkl', 'rb') as fp:
    int2en = pickle.load(fp)
      

In [0]:
# Reading a list of tokens for further usage 
# SQL tokens

with open ('int2sql.pkl', 'rb') as fp:
  int2sql = pickle.load(fp)

In [37]:
type(int2en), type(int2sql)

(list, list)

In [36]:
# Creating indices

sql2int = defaultdict(lambda:3, {n:k for k,n in enumerate(int2sql)})
en2int = defaultdict(lambda:3, {n:k for k,n in enumerate(int2en)})



len(sql2int), len(en2int)

(688, 859)

In [39]:

sql_ids = np.array([[0]+[sql2int[o] for o in sent]+[2] for sent in sql_toks])
en_ids = np.array([[0]+[en2int[o] for o in sent]+[2] for sent in en_toks])
len(sql_ids),len(en_ids), sql_ids[3],en_ids[3]

(4350,
 4350,
 [0,
  25,
  12,
  24,
  41,
  13,
  49,
  51,
  6,
  50,
  54,
  6,
  19,
  22,
  6,
  9,
  23,
  6,
  8,
  17,
  6,
  9,
  33,
  6,
  8,
  30,
  14,
  69,
  4,
  10,
  12,
  148,
  10,
  69,
  11,
  13,
  49,
  51,
  6,
  50,
  54,
  6,
  19,
  22,
  6,
  9,
  23,
  6,
  8,
  17,
  6,
  9,
  33,
  6,
  8,
  30,
  14,
  114,
  4,
  7,
  115,
  7,
  5,
  41,
  4,
  52,
  5,
  53,
  4,
  18,
  5,
  58,
  4,
  7,
  116,
  7,
  5,
  10,
  27,
  4,
  21,
  5,
  20,
  4,
  15,
  5,
  16,
  4,
  7,
  48,
  7,
  5,
  26,
  4,
  31,
  5,
  32,
  4,
  28,
  5,
  29,
  4,
  7,
  59,
  7,
  11,
  11,
  5,
  41,
  4,
  52,
  5,
  53,
  4,
  18,
  5,
  58,
  4,
  7,
  116,
  7,
  5,
  10,
  27,
  4,
  21,
  5,
  20,
  4,
  15,
  5,
  16,
  4,
  7,
  48,
  7,
  5,
  26,
  4,
  31,
  5,
  32,
  4,
  28,
  5,
  29,
  4,
  7,
  59,
  7,
  11,
  2],
 [0, 9, 23, 7, 268, 169, 67, 70, 40, 5, 12, 4, 20, 8, 68, 31, 2])

In [0]:
# we set seed for reproducibility/splitting train and test

np.random.seed(42)

In [72]:
#Padding for ebatch normalization

import keras
from keras.preprocessing.sequence import pad_sequences

englen_90 = int(np.percentile([len(o) for o in en_ids], 99))
sqllen_90 = int(np.percentile([len(o) for o in sql_ids], 99))
englen_90, sqllen_90

(27, 254)

In [0]:
sql_ids = pad_sequences(sql_ids, maxlen=254, dtype='int32', padding='post', truncating='post', value=1)
en_ids = pad_sequences(en_ids, maxlen=27, dtype='int32', padding='post', truncating='post', value=1)

In [74]:
trn_keep = np.random.rand(len(en_ids))>0.1
en_trn,sql_trn = en_ids[trn_keep],sql_ids[trn_keep]
en_val,sql_val = en_ids[~trn_keep],sql_ids[~trn_keep]
len(en_trn),len(en_val)

(3886, 464)

In [0]:
# Using pytorch framework to train seq2seq

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd.variable import Variable
from torch.utils.data import Dataset,DataLoader


In [0]:
from numpy import array as A

In [0]:
#Creating a seq2seq

class seq2seqDataset(Dataset):
  def __init__(self, x, y): self.x,self.y = x,y
  def __getitem__(self, idx): return A(self.x[idx]), A(self.y[idx])
  def __len__(self): return len(self.x)
  
  

In [0]:
# Training sequence

trn_ds = seq2seqDataset(en_trn,sql_trn)
val_ds = seq2seqDataset(en_val,sql_val)

bs = 120 #batch size

# Loading training dataset

trn_dl = DataLoader(trn_ds,batch_size=bs,shuffle=True)
val_dl = DataLoader(val_ds,batch_size=bs)

In [79]:
x, y = next(iter(val_dl))
x.size(), y.size()



(torch.Size([120, 27]), torch.Size([120, 254]))

In [0]:
## load fasttext vectors
#code from here:https://github.com/facebookresearch/fastText/blob/master/docs/crawl-vectors.md
import io
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    header = fin.readline().split()
    n, d = int(header[0]), int(header[1])
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(tokens[1:], dtype=float)
    return data, int(n), int(d)

In [0]:
# get word vectors
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
!unzip wiki-news-300d-1M.vec.zip
!mv wiki-news-300d-1M.vec data/


--2019-08-18 08:14:16--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip.1’

wiki-news-300d-1M.v 100%[===================>] 650.22M  38.5MB/s    in 18s     

2019-08-18 08:14:34 (37.1 MB/s) - ‘wiki-news-300d-1M.vec.zip.1’ saved [681808098/681808098]

Archive:  wiki-news-300d-1M.vec.zip
replace wiki-news-300d-1M.vec? [y]es, [n]o, [A]ll, [N]one, [r]ename: 